<a href="https://colab.research.google.com/github/Abhiramsaigoud/2203A51272-B18/blob/main/Assignment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use a small dataset of English to French sentence pairs. You can replace this with any other language pair dataset.
data = [ ("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"), ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"), ("thank you", "merci"), ("goodbye", "au revoir") ] [CO5]
(a) Take input (English) and target (French)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    print(f"Example {i+1}:")
    print(f"Input: {input_text}")
    print(f"Target: {target_text}")
    print()

Example 1:
Input: hello
Target: 	bonjour


Example 2:
Input: how are you
Target: 	comment ça va


Example 3:
Input: I am fine
Target: 	je vais bien


Example 4:
Input: what is your name
Target: 	comment tu t'appelles


Example 5:
Input: my name is
Target: 	je m'appelle


Example 6:
Input: thank you
Target: 	merci


Example 7:
Input: goodbye
Target: 	au revoir




(b) Building the Seq2Seq Model

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_seq = input_tokenizer.texts_to_sequences(input_texts)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts)
target_seq = target_tokenizer.texts_to_sequences(target_texts)

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

max_input_length = max(len(seq) for seq in input_seq)
max_target_length = max(len(seq) for seq in target_seq)

input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')
target_seq = pad_sequences(target_seq, maxlen=max_target_length, padding='post')

decoder_input_seq = target_seq[:, :-1]
decoder_output_seq = target_seq[:, 1:]

decoder_output_seq_onehot = np.zeros((len(decoder_output_seq), max_target_length - 1, target_vocab_size), dtype="float32")
for i, seq in enumerate(decoder_output_seq):
    for t, word in enumerate(seq):
        decoder_output_seq_onehot[i, t, word - 1] = 1.0

encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_target_length - 1,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2)

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22            │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_23            │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_12 (Embedding)  │ (None, 4, 256)         │          3,840 │ input_layer_22[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_13 (Embedding)  │ (None, 2, 256)         │          3,584 │ input_layer_23[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_12 (LSTM)            │ [(None, 256), (None,   │        525,312 │ embedding_12[0][0]     │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_13 (LSTM)            │ [(None, 2, 256),       │        525,312 │ embedding_13[0][0],    │
│                           │ (None, 256), (None,    │                │ lstm_12[0][1],         │
│                           │ 256)]                  │                │ lstm_12[0][2]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 2, 14)          │          3,598 │ lstm_13[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,061,646 (4.05 MB)

 Trainable params: 1,061,646 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 267ms/step - accuracy: 0.0000e+00 - loss: 2.6413 - val_accuracy: 0.7500 - val_loss: 2.6035
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7750 - loss: 2.5561 - val_accuracy: 0.7500 - val_loss: 2.5166
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5938 - loss: 2.4579 - val_accuracy: 0.7500 - val_loss: 2.3767
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4062 - loss: 2.3686 - val_accuracy: 0.7500 - val_loss: 2.2125
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4062 - loss: 2.1943 - val_accuracy: 0.7500 - val_loss: 1.9152
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.3750 - loss: 1.9901 - val_accuracy: 0.7500 - val_loss: 1.6037
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.3750 - loss: 1.7685 - val_accuracy: 0.7500 - val_loss: 1.6595
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 1.3972 - val_accuracy: 0.7500 - val_loss: 2.

In [ ]:
# Train the Seq2Seq model
model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2)


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 4.9576e-04 - val_accuracy: 0.5000 - val_loss: 5.5730
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 1.0000 - loss: 4.6348e-04 - val_accuracy: 0.5000 - val_loss: 5.5758
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 1.0000 - loss: 4.1785e-04 - val_accuracy: 0.5000 - val_loss: 5.5784
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 3.7760e-04 - val_accuracy: 0.5000 - val_loss: 5.5810
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 4.1588e-04 - val_accuracy: 0.5000 - val_loss: 5.5834
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 1.0000 - loss: 3.6099e-04 - val_accuracy: 0.5000 - val_loss: 5.5862
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 4.2864e-04 - val_accuracy: 0.5000 - val_loss: 5.5889
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 1.0000 - loss: 4.2148e-04 - val_accu

(d) After training, set up separate models for the encoder and decoder to perform inference (translation) on new sentences.

In [ ]:
# Decoder's input for inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Decoder LSTM with stateful behavior
decoder_lstm_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

# Dense layer for prediction
decoder_output = decoder_dense(decoder_lstm_output)

# Define decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_output] + decoder_states)


(c) Prepare the target sequences for training by shifting them by one position, which the model will use to predict the next word in the sequence

In [ ]:
def translate(input_text):

    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_tokenizer.word_index['\t']

    translated_sentence = ''

    for _ in range(max_target_length - 1):

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        sampled_word = target_tokenizer.index_word.get(sampled_token_index, '')

        print(f"Sampled token index: {sampled_token_index}, word: {sampled_word}")

        if sampled_word == '\n':
            break

        translated_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return translated_sentence.strip()

input_sentence = "hello"
translated_sentence = translate(input_sentence)

print(f"Input: {input_sentence}")
print(f"Translation: {translated_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Sampled token index: 15, word: 

Input: hello
Translation: 


(e) Use the trained model to translate new English sentences into French

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_seq = input_tokenizer.texts_to_sequences(input_texts)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts)
target_seq = target_tokenizer.texts_to_sequences(target_texts)

target_tokenizer.word_index['\t'] = len(target_tokenizer.word_index) + 1
target_tokenizer.word_index['\n'] = len(target_tokenizer.word_index) + 1

target_tokenizer.index_word[target_tokenizer.word_index['\t']] = '\t'
target_tokenizer.index_word[target_tokenizer.word_index['\n']] = '\n'

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

max_input_length = max(len(seq) for seq in input_seq)
max_target_length = max(len(seq) for seq in target_seq)

input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')
target_seq = pad_sequences(target_seq, maxlen=max_target_length, padding='post')

decoder_input_seq = target_seq[:, :-1]
decoder_output_seq = target_seq[:, 1:]

decoder_output_seq_onehot = np.zeros((len(decoder_output_seq), max_target_length - 1, target_vocab_size), dtype="float32")
for i, seq in enumerate(decoder_output_seq):
    for t, word in enumerate(seq):
        decoder_output_seq_onehot[i, t, word - 1] = 1.0

encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_target_length-1,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2, callbacks=[early_stopping])

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_output = decoder_dense(decoder_lstm_output)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_output] + decoder_states)

def translate(input_text):

    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_tokenizer.word_index['\t']

    translated_sentence = ''

    for _ in range(max_target_length - 1):

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_tokenizer.index_word[sampled_token_index]

        if sampled_word == '\n':
            break

        translated_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return translated_sentence.strip()

for input_sentence, target_sentence in zip(input_texts, target_texts):
    translated_sentence = translate(input_sentence)
    print(f"Input (English): {input_sentence}")
    print(f"Actual Translation (French): {target_sentence.strip()}")
    print(f"Translated Sentence (French): {translated_sentence}")
    print('-' * 40)

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24            │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_25            │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_14 (Embedding)  │ (None, 4, 256)         │          3,840 │ input_layer_24[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_15 (Embedding)  │ (None, 2, 256)         │          4,096 │ input_layer_25[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_14 (LSTM)            │ [(None, 256), (None,   │        525,312 │ embedding_14[0][0]     │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_15 (LSTM)            │ [(None, 2, 256),       │        525,312 │ embedding_15[0][0],    │
│                           │ (None, 256), (None,    │                │ lstm_14[0][1],         │
│                           │ 256)]                  │                │ lstm_14[0][2]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 2, 16)          │          4,112 │ lstm_15[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,062,672 (4.05 MB)

 Trainable params: 1,062,672 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 284ms/step - accuracy: 0.0500 - loss: 2.7666 - val_accuracy: 0.7500 - val_loss: 2.6583
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5125 - loss: 2.6579 - val_accuracy: 0.7500 - val_loss: 2.5551
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5000 - loss: 2.5449 - val_accuracy: 0.7500 - val_loss: 2.4124
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.3250 - loss: 2.4806 - val_accuracy: 0.7500 - val_loss: 2.1990
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.3688 - loss: 2.1793 - val_accuracy: 0.7500 - val_loss: 1.7929
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.4313 - loss: 1.7852 - val_accuracy: 0.7500 - val_loss: 1.4430
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.1813 - loss: 1.9705 - val_accuracy: 0.7500 - val_loss: 1.5248
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.3375 - loss: 1.4872 - val_accuracy: 0.7500 - val_loss: 1.8059